In [ ]:
import os
import pandas as pd

In [ ]:
columns = "E,I:K"
reimbursed_medicines_path = os.path.join("data", "Refundowane.xlsx")
reimbursed_medicines_df = pd.read_excel(reimbursed_medicines_path, sheet_name="A1", skiprows=1, usecols=columns)
reimbursed_medicines_df = reimbursed_medicines_df.rename(columns={"Numer GTIN lub inny kod jednoznacznie identyfikujący produkt" : "GTIN"})
display(reimbursed_medicines_df)

In [ ]:
rpl_path = os.path.join("data", "RPL_20240102.csv")
cols_to_drop=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 17, 18, 19, 20, 21, 22, 23, 24, 25]

rpl_df = pd.read_csv(rpl_path, sep=";")
rpl_df.drop(rpl_df.columns[cols_to_drop], inplace=True, axis=1)
display(rpl_df)

In [ ]:
rpl_df["Opakowanie"] = rpl_df["Opakowanie"].str.replace("\n", " ")
rpl_df["GTIN"] = rpl_df["Opakowanie"].str.extractall(r"(\d{12,14}|\d{8})").groupby(level=0)[0].apply(list)
rpl_df = rpl_df.explode("GTIN")
rpl_df.drop("Opakowanie", axis=1, inplace=True)
rpl_df.dropna(subset=["GTIN"], inplace=True)
rpl_df["GTIN"] = rpl_df["GTIN"].astype(int)

In [ ]:
merged_df = pd.merge(reimbursed_medicines_df, rpl_df, how="left", left_on="GTIN", right_on="GTIN")
merged_df